In [ ]:
%pip  install llama-index-experimental
%pip install llama-index-llms-groq
%pip install python-dotenv


In [24]:
from llama_index.core.query_pipeline import (
    QueryPipeline as QP,
    Link,
    InputComponent,
)
from llama_index.experimental.query_engine.pandas import (
    PandasInstructionParser,
)
from llama_index.core import PromptTemplate
import pandas as pd
from llama_index.llms.groq import Groq
import os
from dotenv import load_dotenv

load_dotenv('/content/drive/MyDrive/.env')

True

In [25]:
# Cria um DataFrame pandas com dados de exemplo.

df = pd.DataFrame({
    'Nome': ['Alice', 'Bob', 'Charlie', 'David'],
    'Idade': [25, 30, 35,2]})

In [27]:
# Define uma string de instrução para converter consultas em código Python executável usando Pandas.

instruction_str = (
    "1. Convert the query to executable Python code using Pandas.\n"
    "2. The final line of code should be a Python expression that can be called with the `eval()` function.\n"
    "3. The code should represent a solution to the query.\n"
    "4. PRINT ONLY THE EXPRESSION.\n"
    "5. Do not quote the expression.\n"
)

# Define um template qpde prompt para interagir com o DataFrame pandas.

pandas_prompt_str = (
    "You are working with a pandas dataframe in Python.\n"
    "The name of the dataframe is `df`.\n"
    "This is the result of `print(df.head())`:\n"
    "{df_str}\n\n"
    "Follow these instructions:\n"
    "{instruction_str}\n"
    "Query: {query_str}\n\n"
    "Expression:"
)

# Define um template de prompt para sintetizar uma resposta dos resultados da consulta.

response_synthesis_prompt_str = (
    "Given an input question, synthesize a response from the query results.\n"
    "Query: {query_str}\n\n"
    "Pandas Instructions (optional):\n{pandas_instructions}\n\n"
    "Pandas Output: {pandas_output}\n\n"
    "Response: "
)

# Cria um PromptTemplate para o prompt do pandas.
pandas_prompt = PromptTemplate(pandas_prompt_str).partial_format(
    instruction_str=instruction_str, df_str=df.head(5)
)

# Cria um PandasInstructionParser para analisar as instruções do pandas.
pandas_output_parser = PandasInstructionParser(df)

# Cria um PromptTemplate para o prompt de síntese de resposta.
response_synthesis_prompt = PromptTemplate(response_synthesis_prompt_str)

# Inicializa um modelo de linguagem com Groq.
llm = Groq(model="llama3-70b-8192", api_key=os.getenv("API_KEY_GROQ"))

In [29]:
# Cria um QueryPipeline com os módulos necessários.

qp = QP(
    modules={
        "input": InputComponent(),
        "pandas_prompt": pandas_prompt,
        "llm1": llm,
        "pandas_output_parser": pandas_output_parser,
        "response_synthesis_prompt": response_synthesis_prompt,
        "llm2": llm,
    },
    verbose=True,
)

# Adiciona uma cadeia ao QueryPipeline para processar a consulta.

qp.add_chain(["input", "pandas_prompt", "llm1", "pandas_output_parser"])

# Adiciona links ao QueryPipeline para conectar os módulos.

qp.add_links(
    [
        Link("input", "response_synthesis_prompt", dest_key="query_str"),
        Link(
            "llm1", "response_synthesis_prompt", dest_key="pandas_instructions"
        ),
        Link(
            "pandas_output_parser",
            "response_synthesis_prompt",
            dest_key="pandas_output",
        ),
    ]
)

# Adiciona um link do prompt de síntese de resposta para llm2.
qp.add_link("response_synthesis_prompt", "llm2")

In [ ]:
# Executa o QueryPipeline com uma consulta de exemplo.
response = qp.run(
    query_str="Qual a população da cidade de Mumbai?",
)

In [ ]:
# Executa o QueryPipeline com uma consulta de exemplo.

print(response.message.content)